In [1]:
!pip install datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and 

In [2]:
import os
import torch
import evaluate
import numpy as np
import numpy as np
import pandas as pd
import torch.nn as nn
from torch.utils.data import DataLoader
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from transformers import AutoConfig, AutoModelForSequenceClassification, PreTrainedTokenizerBase, Trainer, TrainingArguments

In [3]:
os.environ["WANDB_DISABLED"] = "true"

In [4]:
df = pd.read_csv('/content/cleaned_data.csv')
df

,type,domain_token_count,SymbolCount_Domain,CharacterContinuityRate,NumberofDotsinURL,SymbolCount_URL,NumberRate_AfterPath,domainlength,Entropy_Afterpath,argPathRatio,...,Entropy_Extension,delimeter_path,Domain_Symbol_Density,URL_Symbol_Density,Combined_Entropy,Dot_Ratio,Number_Rate_Ratio,Overall_Complexity,Total_Symbol_Count,Path_to_URL_Ratio
0,1,0.093023,0.069767,0.068911,0.095238,0.021505,0.0,0.105932,0.470801,0.0,...,0.308309,0.025641,0.658605,0.163448,0.389555,1.023810,0.0,0.022970,0.096248,0.0
1,1,0.093023,0.069767,0.050851,0.119048,0.026882,0.0,0.105932,0.531055,0.0,...,0.000000,0.051282,0.658605,0.170988,0.265528,1.279762,0.0,0.016950,0.101624,0.0
2,1,0.093023,0.069767,0.044576,0.095238,0.026882,0.0,0.105932,0.522131,0.0,...,0.000000,0.076923,0.658605,0.147011,0.261065,1.023810,0.0,0.014859,0.096649,0.0
3,1,0.093023,0.069767,0.045276,0.095238,0.026882,0.0,0.105932,0.516925,0.0,...,0.000000,0.076923,0.658605,0.147011,0.258463,1.023810,0.0,0.015092,0.096649,0.0
4,1,0.093023,0.069767,0.026628,0.095238,0.045699,0.0,0.105932,0.558121,0.0,...,0.000000,0.076923,0.658605,0.249919,0.279061,1.023810,0.0,0.008876,0.150292,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
653041,3,0.000000,0.000000,0.024744,0.071429,0.016129,0.0,0.000000,0.592754,0.0,...,0.308309,0.076923,0.000000,0.209677,0.450531,0.000000,0.0,0.008248,0.021104,0.0
653042,3,0.000000,0.000000,0.021828,0.047619,0.021505,0.0,0.000000,0.577480,0.0,...,0.000000,0.102564,0.000000,0.209677,0.288740,0.000000,0.0,0.007276,0.021505,0.0
653043,3,0.000000,0.000000,0.070574,0.047619,0.016129,0.0,0.000000,0.564499,0.0,...,0.000000,0.102564,0.000000,0.157258,0.282250,0.000000,0.0,0.023525,0.016129,0.0
653044,3,0.000000,0.000000,0.021322,0.047619,0.024194,0.0,0.000000,0.558292,0.0,...,0.000000,0.051282,0.000000,0.471774,0.279146,0.000000,0.0,0.007107,0.049069,0.0


In [5]:
features = df.drop('type', axis=1)
labels = df['type']

X_train, X_val, y_train, y_val = train_test_split(features, labels, test_size=0.2, random_state=42, stratify=labels)

In [6]:
train_dataset = Dataset.from_dict({
    'features': X_train.values.tolist(),
    'label': y_train.values.tolist()
})

val_dataset = Dataset.from_dict({
    'features': X_val.values.tolist(),
    'label': y_val.values.tolist()
})

dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset
})

In [7]:
num_features = len(features.columns)
num_labels = len(labels.unique())

config = AutoConfig.from_pretrained(
    "bert-base-uncased",
    num_labels=num_labels,
    hidden_size=128,
    num_hidden_layers=4,
    num_attention_heads=4,
    intermediate_size=256,
)

model = AutoModelForSequenceClassification.from_config(config)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [8]:
class NumericalTokenizer:
    def __call__(self, features, padding='max_length', truncation=True, **kwargs):
        features_tensor = torch.tensor(features, dtype=torch.float32)
        attention_mask = torch.ones_like(features_tensor)

        return {
            'input_values': features_tensor,
            'attention_mask': attention_mask
        }

tokenizer = NumericalTokenizer()

In [9]:
def tokenize_function(examples):
    return tokenizer(examples['features'])

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/522436 [00:00<?, ? examples/s]

Map:   0%|          | 0/130610 [00:00<?, ? examples/s]

In [10]:
class NumericalTransformer(nn.Module):
    def __init__(self, input_size, num_labels):
        super().__init__()
        self.dense = nn.Linear(input_size, 128)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(128, num_labels)

    def forward(self, input_values=None, attention_mask=None, labels=None):
        x = self.dense(input_values)
        x = torch.relu(x)
        x = self.dropout(x)
        logits = self.classifier(x)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.classifier.out_features),
                           labels.view(-1))

        return {'loss': loss, 'logits': logits}

model = NumericalTransformer(input_size=num_features, num_labels=num_labels)

In [11]:
class NumericalTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        loss = outputs['loss']
        return (loss, outputs) if return_outputs else loss

In [12]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=15,
    learning_rate=1e-5,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=15,
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none"
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,29.308400,13.066702,0.706791
2,6.877400,11.903630,0.717579
3,2.476300,6.750619,0.738649
4,6.175300,4.442595,0.765140
5,7.032000,2.907445,0.777498
6,1.496900,2.035506,0.827195
7,0.956500,1.832603,0.827418
8,0.591300,1.545087,0.808024
9,2.462400,1.274746,0.827953
10,0.652900,1.185414,0.819730


TrainOutput(global_step=979575, training_loss=7.568843402323684, metrics={'train_runtime': 4875.1134, 'train_samples_per_second': 1607.458, 'train_steps_per_second': 200.934, 'total_flos': 0.0, 'train_loss': 7.568843402323684, 'epoch': 15.0})

In [14]:
results = trainer.evaluate()
print(results)

{'eval_loss': 0.907996654510498, 'eval_accuracy': 0.8267284281448587, 'eval_runtime': 38.2936, 'eval_samples_per_second': 3410.754, 'eval_steps_per_second': 426.364, 'epoch': 15.0}
